## Air quality in Qatar and other world cup host countries

#### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen
import json

In [5]:
import tabula
from tabula.io import read_pdf

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

### Get list of host countries, past and future

In [7]:
bids_and_hosts = pd.read_html("https://en.wikipedia.org/wiki/FIFA_World_Cup_hosts")[8]

In [8]:
hosts = (
    bids_and_hosts[bids_and_hosts["Times hosted"] > 0]
    .value_counts("Country")
    .reset_index()["Country"]
)

In [9]:
freq_hosts = (
    bids_and_hosts[bids_and_hosts["Times hosted"] > 1]
    .value_counts("Country")
    .reset_index()["Country"]
)

#### countries that could have hosted this year + qatar

In [10]:
bids_2022 = pd.read_html("https://en.wikipedia.org/wiki/FIFA_World_Cup_hosts")[6]

In [11]:
bids = ["United States", "South Korea", "Japan", "Australia"]

### Comparing Qatar air quality with past, future and possible hosts

In [12]:
datafile = urlopen(
    "http://berkeleyearth.lbl.gov/air-quality/maps/cities/country_averages.json"
)

In [13]:
data = json.load(datafile)

In [14]:
rows = []
countries = []
hour = []
day = []
twodays = []
threedays = []
fivedays = []
week = []
month = []
year = []
who = []

for i in range(0, 92):
    rows.append(data["Countries"][i])
    countries.append(data["Countries"][i][0])
    hour.append(data["Countries"][i][1])
    day.append(data["Countries"][i][2])
    twodays.append(data["Countries"][i][3])
    threedays.append(data["Countries"][i][4])
    fivedays.append(data["Countries"][i][5])
    week.append(data["Countries"][i][6])
    month.append(data["Countries"][i][7])
    year.append(data["Countries"][i][8])
    who.append(data["Countries"][i][9])

In [15]:
aq_df = pd.DataFrame(
    {
        "country": countries,
        "last_hour": hour,
        "last_day": day,
        #        "last_two_days": twodays,
        #        "last_three_days": threedays,
        #        "last_five_days": fivedays,
        "last_week": week,
        "last_month": month,
        "last_year": year,
        "whoguidelines_lastyear": who,
    }
)

In [57]:
aq_long = (
    aq_df[(aq_df["country"].isin(hosts))]
    .melt(
        id_vars=["country"],
        value_vars=[
            "last_hour",
            "last_day",
            #            "last_two_days",
            #            "last_three_days",
            #            "last_five_days",
            "last_week",
            "last_month",
            "last_year",
        ],
    )
    .copy()
)

In [58]:
missing_data = (
    aq_long[aq_long["value"].isna()].value_counts("country").reset_index()["country"]
)

In [59]:
aq_long = aq_long[~aq_long["country"].isin(missing_data)].copy()

In [60]:
aq_heatmap = pd.DataFrame(
    {
        "x": aq_long["variable"].ravel(),
        "y": aq_long["country"].ravel(),
        "z": aq_long["value"].ravel(),
    }
)

In [63]:
alt.Chart(aq_heatmap).mark_rect().encode(
    x=alt.X("x", axis=alt.Axis(title="")),
    y=alt.Y("y", axis=alt.Axis(title="")),
    color=alt.Color(
        "z:Q",
        legend=alt.Legend(title="PM2.5 particles"),
        scale=alt.Scale(scheme="spectral", reverse=True),
    ),
).properties(width=360, height=220)

alt.Chart(...)

### Qatar hourly data

In [21]:
qatar_aq = (
    pd.read_csv(
        "http://berkeleyearth.lbl.gov/air-quality/maps/cities/Qatar/Qatar.txt",
        skiprows=7,
        sep="\t",
    )
    .assign(country="Qatar")
    .reset_index()
)

In [22]:
qatar_aq.columns = [
    "year",
    "month",
    "day",
    "utc_hour",
    "pm2.5",
    "pm10_mask",
    "retrospective",
    "country",
]

In [23]:
qatar_aq["date"] = pd.to_datetime(
    qatar_aq["year"].astype(str)
    + "-"
    + qatar_aq["month"].astype(str)
    + "-"
    + qatar_aq["day"].astype(str)
)

In [24]:
qatar_daily_aq = qatar_aq.groupby(["month", "day"])["pm2.5"].mean().reset_index().copy()

#### heatmap of qatar air quality

In [25]:
qatar_src = pd.DataFrame(
    {
        "x": qatar_daily_aq["month"].ravel(),
        "y": qatar_daily_aq["day"].ravel(),
        "z": qatar_daily_aq["pm2.5"].ravel(),
    }
)

In [26]:
alt.Chart(qatar_src).mark_rect().encode(
    x=alt.X(
        "x:O",
    ),
    y=alt.Y("y:O"),
    color=alt.Color(
        "z:Q",
        legend=alt.Legend(title="PM2.5 particles"),
        scale=alt.Scale(scheme="spectral", reverse=True, domain=[0, 91]),
    ),
)

/Users/amatthews/.local/share/virtualenvs/notebooks-tkHVtsX-/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### Get hourly data for bid countries and frequent hosts

In [27]:
for_links = freq_hosts.str.replace(" ", "_")[1:]

In [28]:
bids = ["South_Korea", "Japan", "Australia"]

In [29]:
dataframes = []
for f in for_links:
    dataframes.append(
        pd.read_csv(
            "http://berkeleyearth.lbl.gov/air-quality/maps/cities/"
            + str(f)
            + "/"
            + str(f)
            + ".txt",
            skiprows=7,
            sep="\t",
        )
        .assign(country=f)
        .reset_index()
    )

In [30]:
for bid in bids:
    dataframes.append(
        pd.read_csv(
            "http://berkeleyearth.lbl.gov/air-quality/maps/cities/"
            + str(bid)
            + "/"
            + str(bid)
            + ".txt",
            skiprows=7,
            sep="\t",
        )
        .assign(country=bid)
        .reset_index()
    )

In [31]:
for d in dataframes:
    d.columns = [
        "year",
        "month",
        "day",
        "utc_hour",
        "pm2.5",
        "pm10_mask",
        "retrospective",
        "country",
    ]

In [32]:
for d in dataframes:
    d["date"] = pd.to_datetime(
        d["year"].astype(str)
        + "-"
        + d["month"].astype(str)
        + "-"
        + d["day"].astype(str)
    )

In [33]:
france_aq = dataframes[0]
germany_aq = dataframes[1]
italy_aq = dataframes[2]
mexico_aq = dataframes[3]
us_aq = dataframes[4]
sk_aq = dataframes[5]
japan_aq = dataframes[6]
aus_aq = dataframes[7]

#### Make "[climate stripes](https://showyourstripes.info/s/southamerica/brazil/riodejaneiro)" with hourly/daily aq data on host countries

In [34]:
df = pd.concat([qatar_aq, us_aq, germany_aq, mexico_aq])[
    ["date", "utc_hour", "country", "pm2.5"]
]

In [35]:
df_daily = df.groupby(["date", "country"])["pm2.5"].mean().reset_index().copy()

In [36]:
df_daily["y"] = 2

In [37]:
df_daily["country"] = df_daily["country"].str.replace("_", " ")

In [38]:
aq_stripes = pd.DataFrame(
    {
        "x": df_daily["date"].ravel(),
        "y": df_daily["y"].ravel(),
        "z": df_daily["pm2.5"].ravel(),
        "country": df_daily["country"],
    }
)

In [39]:
alt.Chart(aq_stripes[aq_stripes["x"] >= "2022-06-01"]).mark_bar().encode(
    x=alt.X(
        "x:O",
        axis=alt.Axis(
            title="",
            values=[
                "2022-01-01T00:00:00",
                "2022-04-01T00:00:00",
                "2022-06-14T00:00:00",
                "2022-07-01T00:00:00",
                "2022-07-15T00:00:00",
                "2022-10-01T00:00:00",
                "2022-11-01T00:00:00",
            ],
        ),
    ),
    y=alt.Y("y", title="", axis=alt.Axis(tickCount=0)),
    color=alt.Color(
        "z",
        scale=alt.Scale(scheme="spectral", reverse=True, domain=(0, 200)),
        legend=alt.Legend(title="Average daily air pollution (PM2.5 particles)"),
    ),
    facet=alt.Facet(
        "country",
        columns=1,
        title="",
        sort=["Mexico", "Germany", "United States", "Qatar"],
    ),
).properties(height=100, width=360)

/Users/amatthews/.local/share/virtualenvs/notebooks-tkHVtsX-/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

### make a heatmap instead

In [40]:
aq_daily_heatmap = pd.DataFrame(
    {
        "x": df_daily["date"].ravel(),
        "y": df_daily["country"].ravel(),
        "z": df_daily["pm2.5"].ravel(),
    }
)

In [52]:
alt.Chart(aq_daily_heatmap[aq_daily_heatmap["x"] >= "2022-10-01"]).mark_rect().encode(
    x=alt.X(
        "x:O",
        axis=alt.Axis(
            title="",
            values=[
                "2022-10-01T00:00:00",
                "2022-10-15T00:00:00",
                "2022-11-01T00:00:00",
            ],
        ),
    ),
    y=alt.Y(
        "y:O",
        sort=["Mexico", "Germany", "United States", "Qatar"],
        axis=alt.Axis(title=""),
    ),
    color=alt.Color(
        "z:Q",
        legend=alt.Legend(title="PM2.5 particles"),
        scale=alt.Scale(scheme="spectral", reverse=True),
    ),
).properties(width=360, height=220)

alt.Chart(...)

### the colors aren't working because the scale is so different--trying a different approach

In [53]:
df_daily.head()

,date,country,pm2.5,y
0,2004-01-01,United States,10.068421,2
1,2004-01-02,United States,9.346250,2
2,2004-01-03,United States,8.789167,2
3,2004-01-04,United States,7.270417,2
4,2004-01-05,United States,6.922917,2


In [62]:
step = 20
overlap = 1

alt.Chart(df_daily[df_daily["date"] >= "2022-10-01"], height=step).mark_area(
    interpolate="monotone", fillOpacity=0.8, stroke="lightgray", strokeWidth=0.5
).encode(
    alt.X("date"),
    alt.Y("pm2.5", scale=alt.Scale(), axis=None),
    alt.Fill("pm2.5", legend=None, scale=alt.Scale(scheme="spectral", reverse=True)),
).facet(
    row=alt.Row(
        "country:N", title=None, header=alt.Header(labelAngle=0, labelAlign="right")
    )
).properties(
    bounds="flush"
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor="end"
)

alt.FacetChart(...)

In [67]:
df_daily[(df_daily["country"] == "Qatar") & (df_daily["date"] >= "2022-01-01")]

,date,country,pm2.5,y
12572,2022-01-01,Qatar,11.723750,2
12576,2022-01-02,Qatar,26.747083,2
12580,2022-01-03,Qatar,63.120000,2
12584,2022-01-04,Qatar,30.872083,2
12588,2022-01-05,Qatar,23.454583,2
...,...,...,...,...
13808,2022-11-12,Qatar,21.319583,2
13812,2022-11-13,Qatar,20.689583,2
13816,2022-11-14,Qatar,18.752500,2
13820,2022-11-15,Qatar,21.077917,2
